<a href="https://colab.research.google.com/github/asherzohara/Python/blob/master/quickstarts/Get_started_LiveAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multimodal Live API - Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_LiveAPI.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

**Preview**: The Live API is in preview.

This notebook demonstrates simple usage of the Gemini 2.0 Multimodal Live API. For an overview of new capabilities refer to the [Gemini 2.0 docs](https://ai.google.dev/gemini-api/docs/models/gemini-v2).

This notebook implements a simple turn-based chat where you send messages as text, and the model replies with audio. The API is capable of much more than that. The goal here is to demonstrate with **simple code**.

Some features of the API are not working in Colab, to try them it is recommended to have a look at this [python script](./Get_started_LiveAPI.py) and run it locally.

If you aren't looking for code, and just want to try multimedia streaming use [Live API in Google AI Studio](https://aistudio.google.com/app/live).

The [Next steps](#next_steps) section at the end of this tutorial provides links to additional resources.

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../quickstarts/Get_started.ipynb) notebook.

In [2]:
%pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 5.5 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

The client will pick up your API key from the environment variable.

In [4]:
from google import genai
from google.genai import types
client = genai.Client()

### Select a model

The [Gemini 2.0 Flash Live](https://ai.google.dev/gemini-api/docs/models#live-api) model works with the Live API to enable low-latency bidirectional voice and video interactions with Gemini. The model can process text, audio, and video input, and it can provide text and audio output.

In [5]:
MODEL = "gemini-2.0-flash-live-001"

### Import

Import all the necessary modules.

In [6]:
import asyncio
import base64
import contextlib
import datetime
import os
import json
import wave
import itertools

from IPython.display import display, Audio

from google import genai
from google.genai import types

## Text to Text

The simplest way to use the Live API is as a text-to-text chat interface, but it can do **a lot** more than this.

In [7]:
config={
    "response_modalities": ["TEXT"]
}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  message = "Hello? Gemini are you there?"
  print("> ", message, "\n")
  await session.send_client_content(
        turns={"role": "user", "parts": [{"text": message}]}, turn_complete=True
  )

  # For text responses, When the model's turn is complete it breaks out of the loop.
  turn = session.receive()
  async for chunk in turn:
    if chunk.text is not None:
      print(f'- {chunk.text}')

>  Hello? Gemini are you there? 

- Yes, I am
-  here! How can I help you today?



## Simple text to audio

The simplest way to playback the audio in Colab, is to write it out to a `.wav` file. So here is a simple wave file writer:

In [8]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

The next step is to tell the model to return audio by setting `"response_modalities": ["AUDIO"]` in the `LiveConnectConfig`.  

When you get a response from the model, then you write out the data to a `.wav` file.

In [9]:
config={
    "response_modalities": ["AUDIO"]
}

async def async_enumerate(aiterable):
  n=0
  async for item in aiterable:
    yield n, item
    n+=1


async with client.aio.live.connect(model=MODEL, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send_client_content(
        turns={"role": "user", "parts": [{"text": message}]}, turn_complete=True
    )

    turn = session.receive()
    async for n,response in async_enumerate(turn):
      if response.data is not None:
        wav.writeframes(response.data)

        if n==0:
          print(response.server_content.model_turn.parts[0].inline_data.mime_type)
        print('.', end='')


display(Audio(file_name, autoplay=True))


>  Hello? Gemini are you there? 

audio/pcm;rate=24000
............

## Towards Async Tasks


The real power of the Live API is that it's real time, and interruptable. You can't get that full power in a simple sequence of steps. To really use the functionality you will move the `send` and `recieve` operations (and others) into their own [async tasks](https://docs.python.org/3/library/asyncio-task.html).

Because of the limitations of Colab this tutorial doesn't totally implement the interactive async tasks, but it does implement the next step in that direction:

- It separates the `send` and `receive`, but still runs them sequentially.  
- In the next tutorial you'll run these in separate `async` tasks.


Setup a quick logger to make debugging easier (switch to `setLevel('DEBUG')` to see debugging messages).

In [10]:
import logging

logger = logging.getLogger('Live')
logger.setLevel('INFO')

The class below implements the interaction with the Live API.

In [11]:
class AudioLoop:
  def __init__(self, turns=None,  config=None):
    self.session = None
    self.index = 0
    self.turns = turns
    if config is None:
      config={
          "response_modalities": ["AUDIO"]}
    self.config = config

  async def run(self):
    logger.debug('connect')
    async with client.aio.live.connect(model=MODEL, config=self.config) as session:
      self.session = session

      async for sent in self.send():
        # Ideally send and recv would be separate tasks.
        await self.recv()

  async def _iter(self):
    if self.turns:
      for text in self.turns:
        print("message >", text)
        yield text
    else:
      print("Type 'q' to quit")
      while True:
        text = await asyncio.to_thread(input, "message > ")

        # If the input returns 'q' quit.
        if text.lower() == 'q':
          break

        yield text

  async def send(self):
    async for text in self._iter():
      logger.debug('send')

      # Send the message to the model.
      await self.session.send_client_content(
        turns={"role": "user", "parts": [{"text": text}]}, turn_complete=True
      )
      logger.debug('sent')
      yield text

  async def recv(self):
    # Start a new `.wav` file.
    file_name = f"audio_{self.index}.wav"
    with wave_file(file_name) as wav:
      self.index += 1

      logger.debug('receive')

      # Read chunks from the socket.
      turn = self.session.receive()
      async for n, response in async_enumerate(turn):
        logger.debug(f'got chunk: {str(response)}')

        if response.data is None:
          logger.debug(f'Unhandled server message! - {response}')
        else:
          wav.writeframes(response.data)
          if n == 0:
            print(response.server_content.model_turn.parts[0].inline_data.mime_type)
          print('.', end='')

      print('\n<Turn complete>')

    display(Audio(file_name, autoplay=True))
    await asyncio.sleep(2)


There are 3 methods worth describing here:

**`run` - The main loop**

This method:

- Opens a `websocket` connecting to the Live API.
- Calls the initial `setup` method.
- Then enters the main loop where it alternates between `send` and `recv` until send returns `False`.
- The next tutorial will demonstrate how to stream media and run these asynchronously.

**`send` - Sends input text to the api**

The `send` method collects input text from the user, wraps it in a `client_content` message (an instance of `BidiGenerateContentClientContent`), and sends it to the model.

If the user sends a `q` this method returns `False` to signal that it's time to quit.

**`recv` - Collects audio from the API and plays it**

The `recv` method collects audio chunks in a loop and writes them to a `.wav` file. It breaks out of the loop once the model sends a `turn_complete` method, and then plays the audio.

To keep things simple in Colab it collects **all** the audio before playing it. [Other examples](#next_steps) demonstrate how to play audio as soon as you start to receive it (using `PyAudio`), and how to interrupt the model (implement input and audio playback on separate tasks).

### Run

Run it:


In [12]:
await AudioLoop(['Hello', "What's your name?"]).run()

message > Hello
audio/pcm;rate=24000
.......
<Turn complete>


message > What's your name?
audio/pcm;rate=24000
..........
<Turn complete>


## Working with resumable sessions

Session resumption allows you to return to a previous interaction with the Live API by sending the last session handle you got from the previous session.

When you set your session to be resumable, the session information keeps stored on the Live API for up to 24 hours. In this time window, you can resume the conversation and refer to previous information you have shared with the model.


### Helper functions

Start by creating the helper functions for your resumable interaction with the Live API. It will include:

In [13]:
import asyncio
import traceback
from asyncio.exceptions import CancelledError

last_handle = None

MODEL = "gemini-2.0-flash-live-001"

client = genai.Client()

async def async_enumerate(aiterable):
  n=0
  async for item in aiterable:
    yield n, item
    n+=1


def show_response(response):
    new_handle = None
    if text := response.text:
        print(text, end="")
    else:
      print(response.model_dump_json(indent=2, exclude_none=True))
    if response.session_resumption_update:
        new_handle = response.session_resumption_update.new_handle
    return new_handle


async def clock():
  time = 0
  while True:
    await asyncio.sleep(60)
    time += 1
    print(f"{time}:00")


async def recv(session):
  global last_handle
  try:
    while True:
        async for response in session.receive():
            new_handle = show_response(response)
            if new_handle:
                last_handle = new_handle
  except asyncio.CancelledError:
    pass


async def send(session):
  while True:
      message = await asyncio.to_thread(input, "message > ")
      if message.lower() == "q":
          break
      await session.send_client_content(turns={
          'role': 'user',
          'parts': [{'text': message}]
      })


async def async_main(last_handle=None):
  config = types.LiveConnectConfig.model_validate({
      "response_modalities": ["TEXT"],
      "session_resumption": {
          'handle': last_handle,
      }
  })
  try:
    async with (
        client.aio.live.connect(model=MODEL, config=config) as session,
        asyncio.TaskGroup() as tg
    ):
        clock_task = tg.create_task(clock())
        recv_task = tg.create_task(recv(session))
        send_task = tg.create_task(send(session))
        await send_task
        raise asyncio.CancelledError()
  except asyncio.CancelledError:
      pass
  except ExceptionGroup as EG:
      traceback.print_exception(EG)

Now you can start interacting with the Live API (type `q` to finish the conversation):

In [14]:
await async_main()

message > 你能干什么
{
  "session_resumption_update": {}
}
我是一个大型语言模型，由 Google 训练。我可以执行以下类型的任务：

**信息检索和问答：**

*   **回答各种问题：** 无论是常识性的问题、事实性的问题，还是需要推理的问题，我都可以尝试回答。
*   **提供信息：** 我可以搜索并总结互联网上的信息，为你提供所需的内容。
*   **解释概念：** 我可以解释复杂的概念，使其更容易理解。
*   **定义术语：** 我可以提供各种术语的定义。
*   **提供建议：** 我可以根据你的需求，提供一些建议和指导。

**文本生成和编辑：**

*   **撰写不同类型的文本：** 我可以撰写文章、故事、诗歌、代码、脚本、音乐作品、电子邮件、信件等。
*   **翻译语言：** 我可以将文本从一种语言翻译成另一种语言。
*   **总结文本：** 我可以将长篇文章或文档总结成更短的摘要。
*   **改写文本：** 我可以用不同的措辞和风格改写文本。
*   **校对和编辑文本：** 我可以帮助你检查文本中的拼写、语法和标点错误，并提供改进建议。
*   **生成代码：** 我可以根据你的描述生成代码，例如 Python、JavaScript 等。

**创意和娱乐：**

*   **生成创意文本：** 我可以生成各种创意文本，例如笑话、谜语、口号等。
*   **创作故事和诗歌：** 我可以根据你的提示创作故事和诗歌。
*   **生成音乐：** 我可以根据你的描述生成音乐作品。

**代码相关：**

*   **生成代码片段：** 能够根据描述生成代码片段，用于各种编程任务。
*   **解释代码：** 能够解释代码的功能和逻辑。
*   **调试代码：** 能够帮助你找到代码中的错误。
*   **提供代码建议：** 能够根据你的需求提供代码优化建议。

**其他：**

*   **进行对话：** 我可以和你进行自然的对话，并根据你的反馈进行学习和改进。
*   **执行指令：** 我可以执行简单的指令，例如设置提醒、搜索信息等。
*   **模拟角色：** 我可以模拟不同的角色进行对话。
*   **提供数学计算：** 能够进行简单的数学计算。

**需要注意的是，我也有一些局限性

  + Exception Group Traceback (most recent call last):
  |   File "<ipython-input-13-f0f7a332e03a>", line 68, in async_main
  |     async with (
  |   File "/usr/lib/python3.11/asyncio/taskgroups.py", line 145, in __aexit__
  |     raise me from None
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "<ipython-input-13-f0f7a332e03a>", line 41, in recv
    |     async for response in session.receive():
    |   File "/usr/local/lib/python3.11/dist-packages/google/genai/live.py", line 416, in receive
    |     while result := await self._receive():
    |                     ^^^^^^^^^^^^^^^^^^^^^
    |   File "/usr/local/lib/python3.11/dist-packages/google/genai/live.py", line 497, in _receive
    |     raw_response = await self._ws.recv(decode=False)
    |                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    |   File "/usr/local/lib/python3.11/dist-packages/websockets/asyn

With the session resumption you have the session handle to refer to your previous sessions. In this example, the handle is saved at the `last_handle` variable as below:

In [15]:
last_handle

'CihkdThhZTY4MnBxN3U1ZWUyajhtNnRiNXhrMnh3Y2hpOWU0aTFxZmYx'

Now you can start a new Live API session, but this time pointing to a handle from a previous session. Also, to test you could gather information from the previous session, you will ask the model what was the second question you asked before (in this example, it was "what is the capital of Brazil?"). You can see the Live API recovering that information:

In [ ]:
await async_main(last_handle)

message > 说教育强国，同学们关注什么？想谈什么？
{
  "session_resumption_update": {}
}
如果让同学们谈论“教育强国”，他们可能会关注和想要谈论以下几个方面：

**1. 教育公平与资源分配：**

*   **城乡差距：** 城乡教育资源差距依然存在，同学们会关注如何缩小城乡教育差距，让所有孩子都能享有优质教育。
*   **地域差异：** 不同省份、地区的教育水平存在差异，同学们会关注如何平衡地域间的教育资源分配。
*   **特殊群体：** 残疾学生、贫困学生等特殊群体的教育权益保障，如何让他们更好地融入社会。
*   **教育机会均等：** 讨论优质教育资源如何更公平地分配，避免“掐尖”现象，关注每个学生的个性化发展。

**2. 教育质量与教学改革：**

*   **应试教育与素质教育：** 如何平衡应试压力和素质培养，让同学们在提升学业成绩的同时，也能发展兴趣爱好、培养创新精神。
*   **教学方法改革：** 探讨更有效的教学方法，如项目式学习、探究式学习、合作学习等，激发学习兴趣和主动性。
*   **教材内容更新：** 关注教材内容是否与时俱进，是否能够反映社会发展和科技进步。
*   **教师队伍建设：** 讨论如何提升教师的专业素养和师德水平，让他们成为学生成长路上的引路人。

**3. 未来发展与个人价值：**

*   **专业选择与职业规划：** 如何根据个人兴趣、社会需求和发展趋势，选择合适的专业和职业方向。
*   **创新创业：** 在“大众创业、万众创新”的背景下，同学们会关注如何培养创新精神和创业能力。
*   **终身学习：** 强调终身学习的重要性，鼓励同学们不断学习新知识、掌握新技能，适应快速变化的社会。
*   **社会责任感：** 讨论如何培养同学们的社会责任感和公民意识，让他们在实现个人价值的同时，也能为国家和社会做出贡献。

**4. 教育评价体系：**

*   **“唯分数论”：** 批判过度强调考试成绩的现象，呼吁建立多元化的评价体系，关注学生的综合素质。
*   **升学压力：** 讨论如何缓解升学压力，让同学们能够健康快乐地成长。
*   **综合素质评价：** 探索更加科学合理的综合素质评价体系，全面反映学生的成长过程和发展潜力。

**5. 国际视野与文

## Next steps

<a name="next_steps"></a>

This tutorial just shows basic usage of the Live API, using the Python GenAI SDK.

- If you aren't looking for code, and just want to try multimedia streaming use [Live API in Google AI Studio](https://aistudio.google.com/app/live).
- If you want to see how to setup streaming interruptible audio and video using the Live API see the [Audio and Video input Tutorial](../quickstarts/Get_started_LiveAPI.py).
- If you're interested in the low level details of using the websockets directly, see the [websocket version of this tutorial](../quickstarts/websockets/Get_started_LiveAPI.ipynb).
- Try the [Tool use in the live API tutorial](../quickstarts/Get_started_LiveAPI_tools.ipynb) for an walkthrough of Gemini-2's new tool use capabilities.
- There is a [Streaming audio in Colab example](../quickstarts/websockets/LiveAPI_streaming_in_colab.ipynb), but this is more of a **demo**, it's **not optimized for readability**.
- Other nice Gemini 2.0 examples can also be found in the [Cookbook's 2.0 directory](https://github.com/google-gemini/cookbook/tree/main/gemini-2/), in particular the [video understanding](../quickstarts/Video_understanding.ipynb) and the [spatial understanding](../quickstarts/Spatial_understanding.ipynb) ones.
